In [70]:
# parse dev data function
def ParseJson(data_json):
    paragraphs = data_json['data'][0]['paragraphs']
    idx_list = list()
    context_list = list()
    ques_list = list()
    ans_list = list()
    for para in paragraphs:
        context = para['context']
        idx = para['qas'][0]['id']
        ques = para['qas'][0]['question']
        ans = '|||'.join([ans['text'] for ans in para['qas'][0]['answers']])
        idx_list.append(idx)
        context_list.append(context)
        ques_list.append(ques)
        ans_list.append(ans)
    return idx_list, context_list, ques_list, ans_list

In [71]:
# dev data
import json
dev_file = '/home/t-honli/data/EQnA/dev-v1.1.json' 
dev_data = json.load(open(dev_file, "r"))

In [72]:
import pandas as pd
idx, context, ques, ans = ParseJson(dev_data)
dev_pd = pd.DataFrame({'id':idx, 'context':context, 'question':ques, 'ground_truth':ans}, columns=['id', 'context', 'question', 'ground_truth'])
dev_pd.head(5)

,id,context,question,ground_truth
0,9200ae8372eadcb43e5cfb816f185ef9,Start Up & Rev of a 2006 Acura RL SH - AWD.Thi...,2006 acura rl horsepower,290 HP
1,b7d7518b27e56604c9b868ba25565222,The income tax withholding rate remains at 4.2...,2015 michigan withholding tax rate,4.25 %
2,db5d9db86604d34be584b62c08b00634,The experts at Vidal Sassoon bring you salon c...,5rr hair color,Red
3,92484e0e63e92f1412854e9516b8f8c3,Effects depend on the substance : hydrogen per...,acid that burns through skin,nitric acid|||hydrogen peroxide
4,c8dcb86f8c0a1954ecd7931480b29441,"Actor Nikolaj Coster - Waldau , the man who pl...",actor who plays jamie lannister,Nikolaj Coster|||Waldau


In [123]:
dev_pd[dev_pd['id'] == 'a1ad804e600a055ec3e1529dc65024aa']

,id,context,question,ground_truth,bidaf,prod,bidaf_single,prod_single
63,a1ad804e600a055ec3e1529dc65024aa,DC has a sales tax rate structure that also in...,dc parking tax rate,18 %,10 %:::0.3227|||18 %:::0.5344|||14.5 %:::0.9012,18 %:::0.7989|||meals:::0.0136|||vehicles:::0....,10 %,18 %


In [76]:
# max answer num
max_num = 0
max_ans = ''
def getMaxNum(row):
    global max_num, max_ans
    tmp_num = len(row['ground_truth'].split('|||'))
    if tmp_num > max_num:
        max_num = tmp_num
        max_ans = row['ground_truth']
    
dev_pd.apply(getMaxNum, axis=1)
print (max_num)
print (max_ans)

# bidaf ans
import json
import os
#file = '/home/t-honli/bi-att-flow/out/EQnA/03-07-2017/answer/test-020000.json'
file = '/home/t-honli/bi-att-flow/out/EQnA/no_sent_token/18-07-2017/answer/top3.json'
bidaf = json.load(open(file, "r"))

23
Alaska|||Arizona|||California|||Colorado|||Connecticut|||Delaware|||Hawaii|||Illinois|||Maine|||Maryland|||Massachusetts|||Michigan|||Minnesota|||Montana|||Nevada|||New Hampshire|||New Jersey|||New Mexico|||New York|||Oregon|||Rhode Island|||Vermont|||the District of Columbia


In [77]:
list(bidaf['scores'].values())[:10]

['0.9227|||0.2311|||0.0830',
 '0.4284|||0.3181|||0.6372',
 '0.1095|||0.1645|||0.2476',
 '0.3587|||0.2714|||0.9464',
 '0.8755|||0.1260|||0.0856',
 '0.7898|||0.9697|||0.0234',
 '0.9092|||0.2312|||0.6155',
 '0.2988|||0.7237|||0.1566',
 '0.0929|||0.1480|||0.1455',
 '0.1399|||0.4392|||0.1738']

In [78]:
# no sent_token answer
dev_pd['bidaf'] = dev_pd.apply(lambda row: '|||'.join([ phrase+':::'+score for phrase, score in  zip(bidaf[row['id']].split('|||'), bidaf['scores'][row['id']].split('|||'))          ]), axis=1)

In [79]:
dev_pd.head(5)

,id,context,question,ground_truth,bidaf
0,9200ae8372eadcb43e5cfb816f185ef9,Start Up & Rev of a 2006 Acura RL SH - AWD.Thi...,2006 acura rl horsepower,290 HP,290 HP:::0.5270|||3.5L:::0.8375|||V6:::0.6325
1,b7d7518b27e56604c9b868ba25565222,The income tax withholding rate remains at 4.2...,2015 michigan withholding tax rate,4.25 %,"4.25 %:::0.9626|||$4,000:::0.9243|||for tax ye..."
2,db5d9db86604d34be584b62c08b00634,The experts at Vidal Sassoon bring you salon c...,5rr hair color,Red,gray:::0.6504|||100 percent:::0.7665|||Vidal S...
3,92484e0e63e92f1412854e9516b8f8c3,Effects depend on the substance : hydrogen per...,acid that burns through skin,nitric acid|||hydrogen peroxide,nitric acid:::0.3067|||hydrogen peroxide:::0.8...
4,c8dcb86f8c0a1954ecd7931480b29441,"Actor Nikolaj Coster - Waldau , the man who pl...",actor who plays jamie lannister,Nikolaj Coster|||Waldau,Waldau:::0.3101|||Nikolaj Coster:::0.9732|||Ac...


In [80]:
dev_pd[dev_pd['id'] == '92dc0ac40c712880b02b422dbd023cf2']

,id,context,question,ground_truth,bidaf
159,92dc0ac40c712880b02b422dbd023cf2,"For a hard - boiled egg , start the egg in col...",how long does it take to boil a hard boiled egg,7 - 10,7 - 10 minutes:::0.7651|||cold water and bring...


In [81]:
# prod model
import pandas as pd
col_names = ['id', 'Query', 'Url', 'Answer', 'AnswerTokenList', 'ParaseSpan', 'phrase', 'Label', 'Probability']
prod = pd.read_csv('/home/t-honli/data/EQnA/EQnA_Highlighting_Test_ProdModel.tsv', header=None, sep='\t', names=col_names, dtype=str).fillna('')

print (len(prod))

# 根据answer，query，url生成hash_id
import hashlib
def GetHashCode(context):
    hash = hashlib.md5()
    hash.update(context.encode('utf-8'))
    return hash.hexdigest()
prod['hash_id'] = prod.apply(lambda row: GetHashCode(row['Answer'] + ' ' + row['Query'] + ' ' + row['Url']), axis=1)

prod = prod[prod.apply(lambda row: len(row['phrase'])<32, axis=1)]
print (len(prod))

15560
15454


In [82]:
# 获取hash_id, [(phrase,score)]
multi_phrase_dict = {}
def getPhrase(row):
    if row['hash_id'] not in multi_phrase_dict:
        multi_phrase_dict[row['hash_id']] = list()
    row['Probability'] = '%.4f' % float(row['Probability'])
    multi_phrase_dict[row['hash_id']].append(row['phrase'] + ":::" + row['Probability'])
prod.apply(getPhrase, axis=1)
print (len(prod))

15454


In [83]:
# sort
for idx in list(multi_phrase_dict.keys()):
    multi_phrase_dict[idx].sort(key=lambda x: float(x.split(':')[-1]), reverse=True)

In [84]:
dev_pd['prod'] = dev_pd.apply(lambda row: '|||'.join(multi_phrase_dict[row['id']]), axis=1)

In [132]:
#pd.set_option('max_colwidth', 200)
dev_pd.head(5)

,id,context,question,ground_truth,bidaf,prod,bidaf_single,prod_single
0,9200ae8372eadcb43e5cfb816f185ef9,Start Up & Rev of a 2006 Acura RL SH - AWD.Thi...,2006 acura rl horsepower,290 HP,290 HP:::0.5270|||3.5L:::0.8375|||V6:::0.6325,290 HP:::0.8149|||V6:::0.4651|||AWD.This:::0.3...,290 HP,290 HP
1,b7d7518b27e56604c9b868ba25565222,The income tax withholding rate remains at 4.2...,2015 michigan withholding tax rate,4.25 %,"4.25 %:::0.9626|||$4,000:::0.9243|||for tax ye...","4.25 %:::0.8581|||$4,000:::0.6614|||2014:::0.0...",4.25 %,4.25 %
2,db5d9db86604d34be584b62c08b00634,The experts at Vidal Sassoon bring you salon c...,5rr hair color,Red,gray:::0.6504|||100 percent:::0.7665|||Vidal S...,Red:::0.9268|||gray:::0.9099|||Vidal Sassoon::...,gray,Red
3,92484e0e63e92f1412854e9516b8f8c3,Effects depend on the substance : hydrogen per...,acid that burns through skin,nitric acid|||hydrogen peroxide,nitric acid:::0.3067|||hydrogen peroxide:::0.8...,hydrogen peroxide:::0.4230|||nitric acid:::0.2...,nitric acid,hydrogen peroxide
4,c8dcb86f8c0a1954ecd7931480b29441,"Actor Nikolaj Coster - Waldau , the man who pl...",actor who plays jamie lannister,Nikolaj Coster|||Waldau,Waldau:::0.3101|||Nikolaj Coster:::0.9732|||Ac...,Jaime:::0.7372|||Nikolaj Coster:::0.7275|||Wal...,Waldau,Jaime


In [188]:
# the top1 is not in answers case  EM 472/2021
import string
import re
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

# match
def filterNotTop1Ans(row):
    grounds = row['ground_truth'].split('|||')
    bidaf = row['bidaf'].split('|||')[0].split(':::')[0]
    if normalize_answer(bidaf) not in set(map(lambda x: normalize_answer(x), grounds)):
        return True
    else:
        return False
# cover
def filterNotTop1Ans_cover(row):
    grounds = row['ground_truth'].split('|||')
    bidaf = normalize_answer(row['bidaf'].split('|||')[0].split(':::')[0])
    for ground in map(lambda x: normalize_answer(x), grounds):
        if bidaf in ground or ground in bidaf:
            return False
    return True

# match
def filterProdNotTop1Ans(row):
    grounds = row['ground_truth'].split('|||')
    bidaf = row['prod'].split('|||')[0].split(':::')[0]
    if normalize_answer(bidaf) not in set(map(lambda x: normalize_answer(x), grounds)):
        return True
    else:
        return False
    
# cover
def filterProdNotTop1Ans_cover(row):
    grounds = row['ground_truth'].split('|||')
    prod = normalize_answer(row['prod'].split('|||')[0].split(':::')[0])
    for ground in map(lambda x: normalize_answer(x), grounds):
        if prod in ground or ground in prod:
            return False
    return True

# bidaf match
notTop1index = dev_pd.apply(filterNotTop1Ans, axis=1)
notTop1Ans = dev_pd[notTop1index]
top1index = notTop1index.apply(lambda x: not x)
Top1Ans = dev_pd[top1index]
print ('bidaf not match:', str(len(notTop1Ans))+'/'+str(len(dev_pd)), 1.0*len(notTop1Ans)/len(dev_pd), 1-1.0*len(notTop1Ans)/len(dev_pd))

# bidaf not match
bidaf_notMatch = notTop1index.values

# bidaf cover
notTop1index = dev_pd.apply(filterNotTop1Ans_cover, axis=1)
notTop1Ans = dev_pd[notTop1index]
top1index = notTop1index.apply(lambda x: not x)
Top1Ans = dev_pd[top1index]
print ('bidaf not cover:', str(len(notTop1Ans))+'/'+str(len(dev_pd)), 1.0*len(notTop1Ans)/len(dev_pd), 1-1.0*len(notTop1Ans)/len(dev_pd))
notTop1Ans.to_csv('/home/t-honli/bi-att-flow/out/scripts/data/bidaf_notTop1Ans.tsv', header=True, index=None, columns=['id', 'context', 'question', 'ground_truth', 'bidaf', 'prod'], sep='\t')

# bidaf not cover
bidaf_notCover = notTop1index.values

# bidaf not match - not cover
bidaf_addCover = [ notMatch ^ notCover for notMatch, notCover in zip(bidaf_notMatch, bidaf_notCover)]
dev_pd[bidaf_addCover].to_csv('/home/t-honli/bi-att-flow/out/scripts/data/bidaf_addCover.tsv', header=True, index=None, columns=['id', 'context', 'question', 'ground_truth', 'bidaf', 'prod'], sep='\t')

# bidaf top1 and socre > 0.36
bidaf_cover = Top1Ans.apply(lambda x: float(x['bidaf'].split('|||')[0].split(':::')[1]) > 0.36, axis=1)
print (sum(bidaf_cover) , len(Top1Ans))

# bidaf top1 
bidaf_cover_top1 = top1index.values

# prod match
notTop1index = dev_pd.apply(filterProdNotTop1Ans, axis=1)
notTop1Ans = dev_pd[notTop1index]
top1index = notTop1index.apply(lambda x: not x)
Top1Ans = dev_pd[top1index]
print ('prod not match:', str(len(notTop1Ans))+'/'+str(len(dev_pd)), 1.0*len(notTop1Ans)/len(dev_pd), 1-1.0*len(notTop1Ans)/len(dev_pd))

# prod cover
notTop1index = dev_pd.apply(filterProdNotTop1Ans_cover, axis=1)
notTop1Ans = dev_pd[notTop1index]
top1index = notTop1index.apply(lambda x: not x)
Top1Ans = dev_pd[top1index]
print ('prod not cover:', str(len(notTop1Ans))+'/'+str(len(dev_pd)), 1.0*len(notTop1Ans)/len(dev_pd), 1-1.0*len(notTop1Ans)/len(dev_pd))
notTop1Ans.to_csv('/home/t-honli/bi-att-flow/out/scripts/data/prod_notTop1Ans.tsv', header=True, index=None, columns=['id', 'context', 'question', 'ground_truth', 'bidaf', 'prod'], sep='\t')

# prod top1 and socre > 0.6
prod_cover = Top1Ans.apply(lambda x: float(x['prod'].split('|||')[0].split(':::')[1]) > 0.6, axis=1)
print (sum(prod_cover) , len(Top1Ans))

# prod top1 
prod_cover_top1 = top1index.values

# unlimit cover union
unlimt_union = [bidaf or prod for bidaf, prod in zip(bidaf_cover_top1, prod_cover_top1)]
print ('bidaf and prod union ', sum(unlimt_union), len(unlimt_union))


bidaf not match: 462/2021 0.22859970311726868 0.7714002968827314
bidaf not cover: 252/2021 0.12469074715487383 0.8753092528451262
1006 1769
prod not match: 278/2021 0.1375556655121227 0.8624443344878773
prod not cover: 252/2021 0.12469074715487383 0.8753092528451262
1451 1769
bidaf and prod union  1934 2021


In [187]:
# bidaf cover > 0.36 | prod cover > 0.6 union
bidaf_limit = dev_pd.apply(lambda x: float(x['bidaf'].split('|||')[0].split(':::')[1]) > 0.36, axis=1).values
bidaf_limit = [bidaf and valid for bidaf, valid in zip(bidaf_limit, bidaf_cover_top1)]
print (sum(bidaf_limit))

prod_limit = dev_pd.apply(lambda x: float(x['prod'].split('|||')[0].split(':::')[1]) > 0.6, axis=1).values
prod_limit =  [prod and valid for prod, valid in zip(prod_limit, prod_cover_top1)] 
print (sum(prod_limit))

limit_union = [bidaf or prod for bidaf, prod in zip(bidaf_limit, prod_limit)]
print ('bidaf and prod limit union ', sum(limit_union), len(limit_union))

1006
1451
bidaf and prod limit union  1705 2021


In [57]:
# single top1 origin
import os
from subprocess import Popen, PIPE
work_dir = '/home/t-honli/bi-att-flow'
os.chdir(work_dir)
print (Popen("python squad/evaluate-v1.1.py $HOME/data/EQnA/dev-v1.1.json out/EQnA/03-07-2017/answer/single.json", shell=True, stdout=PIPE, stderr=PIPE).stdout.readlines())

[b'{"exact_match": 76.69470559129144, "f1": 82.9467874158819}\n']


In [68]:
# single top1 no sentence token (used)
import os
from subprocess import Popen, PIPE
work_dir = '/home/t-honli/bi-att-flow'
os.chdir(work_dir)
print (Popen("python squad/evaluate-v1.1.py $HOME/data/EQnA/dev-v1.1.json out/EQnA/no_sent_token/18-07-2017/answer/single.json", shell=True, stdout=PIPE, stderr=PIPE).stdout.readlines())

[b'{"exact_match": 77.14002968827313, "f1": 83.41632514874934}\n']


In [69]:
# single top1 sentence token
import os
from subprocess import Popen, PIPE
work_dir = '/home/t-honli/bi-att-flow'
os.chdir(work_dir)
print (Popen("python squad/evaluate-v1.1.py $HOME/data/EQnA/dev-v1.1.json out/EQnA/sent_token/18-07-2017/answer/single.json", shell=True, stdout=PIPE, stderr=PIPE).stdout.readlines())

[b'{"exact_match": 75.65561603166749, "f1": 82.3026645625083}\n']


In [163]:
# trick test bidaf EM and F1 one slot case
bidaf_single_dict = dict()
def getBidafSingleAnswer(row):
    global bidaf_single_dict
    spans = row['bidaf'].split('|||')
    
    # 只对一个answer的case采用tricks，从第一个开始找，大于阈值就返回
    if len(row['ground_truth'].split('|||') ) == 1:
        for idx, span in enumerate(spans):
            ans, score = span.split(':::')
            if float(score) > 0.5:
                bidaf_single_dict[row['id']] = ans
                return ans
            
    bidaf_single_dict[row['id']] = spans[0].split(':::')[0]
    return spans[0].split(':::')[0]
dev_pd['bidaf_single'] = dev_pd.apply(getBidafSingleAnswer, axis=1)

with open('bidaf_single.json', 'w') as json_file:
    json_file.write(json.dumps(bidaf_single_dict))

# single top1
import os
from subprocess import Popen, PIPE
work_dir = '/home/t-honli/bi-att-flow'
os.chdir(work_dir)
print (Popen("python squad/evaluate-v1.1.py $HOME/data/EQnA/dev-v1.1.json bidaf_single.json", shell=True, stdout=PIPE, stderr=PIPE).stdout.readlines())

[b'{"exact_match": 74.3691241959426, "f1": 80.315737065916}\n']


In [156]:
# limit test prod EM and F1
prod_single_dict = dict()
def getProdSingleAnswer(row):
    global prod_single_dict
    spans = row['prod'].split('|||')
    
    if float(spans[0].split(':::')[1]) > 0.6:
        prod_single_dict[row['id']] = spans[0].split(':::')[0]
        return spans[0].split(':::')[0]
    else:
        prod_single_dict[row['id']] = ''
        return ''
    
dev_pd['prod_single'] = dev_pd.apply(getProdSingleAnswer, axis=1)

with open('prod_single.json', 'w') as json_file:
    json_file.write(json.dumps(prod_single_dict))

# single top1
import os
from subprocess import Popen, PIPE
work_dir = '/home/t-honli/bi-att-flow'
os.chdir(work_dir)
print (Popen("python squad/evaluate-v1.1.py $HOME/data/EQnA/dev-v1.1.json prod_single.json", shell=True, stdout=PIPE, stderr=PIPE).stdout.readlines())

[b'{"exact_match": 71.25185551707075, "f1": 71.93880917037772}\n']


In [164]:
# trick test bidaf EM and F1 add case
bidaf_single_dict = dict()
def getBidafSingleAnswer(row):
    global bidaf_single_dict
    spans = row['bidaf'].split('|||')
    
    # 多所有answer的case采用tricks，从第一个开始找，大于阈值就返回
    for idx, span in enumerate(spans):
        ans, score = span.split(':::')
        if float(score) > 0.36:
            bidaf_single_dict[row['id']] = ans
            return ans
            
    bidaf_single_dict[row['id']] = ''
    return ''
dev_pd['bidaf_single'] = dev_pd.apply(getBidafSingleAnswer, axis=1)

with open('bidaf_single.json', 'w') as json_file:
    json_file.write(json.dumps(bidaf_single_dict))

# single top1
import os
from subprocess import Popen, PIPE
work_dir = '/home/t-honli/bi-att-flow'
os.chdir(work_dir)
print (Popen("python squad/evaluate-v1.1.py $HOME/data/EQnA/dev-v1.1.json bidaf_single.json", shell=True, stdout=PIPE, stderr=PIPE).stdout.readlines())

[b'{"exact_match": 64.42355269668481, "f1": 66.76892627412171}\n']


In [95]:
dev_pd.head(5)

,id,context,question,ground_truth,bidaf,prod,bidaf_single,prod_single
0,9200ae8372eadcb43e5cfb816f185ef9,Start Up & Rev of a 2006 Acura RL SH - AWD.Thi...,2006 acura rl horsepower,290 HP,290 HP:::0.5270|||3.5L:::0.8375|||V6:::0.6325,290 HP:::0.8149|||V6:::0.4651|||AWD.This:::0.3...,290 HP,290 HP
1,b7d7518b27e56604c9b868ba25565222,The income tax withholding rate remains at 4.2...,2015 michigan withholding tax rate,4.25 %,"4.25 %:::0.9626|||$4,000:::0.9243|||for tax ye...","4.25 %:::0.8581|||$4,000:::0.6614|||2014:::0.0...",4.25 %,4.25 %
2,db5d9db86604d34be584b62c08b00634,The experts at Vidal Sassoon bring you salon c...,5rr hair color,Red,gray:::0.6504|||100 percent:::0.7665|||Vidal S...,Red:::0.9268|||gray:::0.9099|||Vidal Sassoon::...,gray,Red
3,92484e0e63e92f1412854e9516b8f8c3,Effects depend on the substance : hydrogen per...,acid that burns through skin,nitric acid|||hydrogen peroxide,nitric acid:::0.3067|||hydrogen peroxide:::0.8...,hydrogen peroxide:::0.4230|||nitric acid:::0.2...,nitric acid,hydrogen peroxide
4,c8dcb86f8c0a1954ecd7931480b29441,"Actor Nikolaj Coster - Waldau , the man who pl...",actor who plays jamie lannister,Nikolaj Coster|||Waldau,Waldau:::0.3101|||Nikolaj Coster:::0.9732|||Ac...,Jaime:::0.7372|||Nikolaj Coster:::0.7275|||Wal...,Waldau,Jaime


In [37]:
dev_pd.to_csv('./sample_multi.tsv', header=True, index=None, sep='\t')

In [38]:
# add score
# SampleAndCompactMulti.ipynb add threshold
# generate_prodformat_label.ipynb change bidaf data to prod format